## 5.1 层和块

多个层被组合成块，形成更大的模型

![](../images/5-1-1.svg)

下面的代码生成了一个多层感知机的网络

In [8]:
import torch 
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0875, -0.0553,  0.1135,  0.0243,  0.0208, -0.0921, -0.0090, -0.0815,
          0.0895, -0.2433],
        [-0.0072,  0.0325,  0.0492, -0.0131, -0.1079, -0.1417,  0.0385, -0.0680,
          0.0913, -0.0665]], grad_fn=<AddmmBackward0>)

实际上，`net(X)`是`net.__call__(X)`的简写

### 5.1.1 自定义块

每个块需要提供如下的基础功能

1. 将输入数据作为其前向传播的参数。
2. 输出 <- 前向传播函数(输入)。
3. 通过自动的方法访问器反向传播函数，并计算其关于输入的梯度。
4. 存储和访问前向传播计算所需的参数(权重)。
5. 根据需要初始化模型参数。

下面从零开始编写一个块

In [9]:
class MLP(nn.Module):
    # 声明模型参数
    def __init__(self):
        super().__init__()
        # 进行层的实例化
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层
    
    # 定义前向传播函数
    def forward(self, X):
        # 使用ReLU函数作为激活函数
        return self.out(F.relu(self.hidden(X)))

测试一下这个函数

In [10]:
net = MLP()
net(X)

tensor([[ 0.0535, -0.2344, -0.0543, -0.1793,  0.0632, -0.0605,  0.0577,  0.0513,
          0.0782, -0.3015],
        [ 0.1266, -0.0984, -0.0521, -0.1705, -0.0242,  0.0376, -0.0883,  0.0132,
         -0.1071, -0.1668]], grad_fn=<AddmmBackward0>)

### 5.1.2 顺序块

构建了一个简单的`Squential`设计。

使用`_module`的主要优点是在模块的参数初始化过程中，系统知道在`_module`字典中查找需要初始化的参数的子块。也就是放在这里的层会被初始化。

In [11]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里的_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

下面使用MySequential实现多层感知机，用法与上面是相同的

In [12]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
net(X)

tensor([[-0.1589, -0.0314,  0.1699,  0.0678,  0.1481, -0.1064, -0.0688, -0.0142,
         -0.0416,  0.1284],
        [-0.1566, -0.0353,  0.1007,  0.0076,  0.2174, -0.1421,  0.0817,  0.0031,
          0.0090, -0.0092]], grad_fn=<AddmmBackward0>)

### 5.1.3 在前向传播函数中执行代码

有时候需要加入一些常数、一些控制流、一些固定的参数，下面给出了一个例子。

需要注意的是，`torch.mm`函数和`torch.matmul`函数有一定的区别，其中`torch.mm`仅能够处理二维的矩阵乘法，而`torch.matmul`可以适用任意维度。

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm参数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，共享参数，这会导致梯度累加
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
net = FixedHiddenMLP()
net(X)

tensor(0.1736, grad_fn=<SumBackward0>)

此外，块还可以嵌套

In [15]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
        )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP(),
)
chimera(X)

tensor(-0.0747, grad_fn=<SumBackward0>)

### 5.1.4 效率

### 5.1.5 小结

- 一个块可以由许多层组成；一个块可以由许多块组成。
- 块可以包含代码。
- 块负责大量的内部处理，包括参数初始化和反向传播。
- 层和块的顺序连接由Sequential块处理。